To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

Features in the notebook:
1. Uses Maxime Labonne's [FineTome 100K](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset.
1. Convert ShareGPT to HuggingFace format via `standardize_sharegpt`
2. Train on Completions / Assistant only via `train_on_responses_only`
3. Unsloth now supports Torch 2.4, all TRL & Xformers versions & Python 3.12!

In [24]:
%%capture
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!apt update && apt install -y cuda-toolkit-11-8
!pip install rank-bm25


# Install unsloth
!pip install unsloth

# Also get the latest nightly Unsloth! obviously
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [25]:
import pandas as pd

# Load the CSV file with the appropriate encoding to handle BOM
df = pd.read_csv('/content/data/Tuning questions.csv', encoding='utf-8-sig')

# Create a list of tuples containing the prompt and response pairs
FILES = [(row['user_prompt'], row['system_response']) for _, row in df.iterrows()]

print(FILES)

[('When does the policy change regarding the CAPS program transition take effect, and to which counties will it apply?', 'The policy change regarding the CAPS program transition took effect on July 17, 2017. Initially, it applied to 13 counties: Bibb, Brooks, Catoosa, Clarke, Colquitt, Cook, Douglas, Echols, Gilmer, Gordon, Lowndes, Murray, and Whitfield.\n\nPlease see the following link for more information: https://fostercaresystems.wustl.edu/foster/childcare-and-parent-services-caps-program-transition-department-early-care-and-learning'), ('What are the responsibilities and standards for caregivers in the Division of Family and Children Services (DFCS) when it comes to making decisions about age and developmentally appropriate activities for children in foster care?', "Caregivers in the Division of Family and Children Services (DFCS) are responsible for ensuring that children in foster care have ongoing opportunities to engage in age and developmentally appropriate activities, inclu

In [26]:
#yessss
from transformers import AutoTokenizer

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/data/Tuning questions.csv')


# FILES = [f"{row['user_prompt']}" for _, row in df.iterrows()]

FILES = [(row['user_prompt'].replace('data/', '').strip(),
          row['system_response'].replace('data/', '').strip())
         for _, row in df.iterrows()
         if isinstance(row['user_prompt'], str) and len(row['user_prompt'].strip()) > 0 and
            isinstance(row['system_response'], str) and len(row['system_response'].strip()) > 0]

print(FILES)
INSTRUCTION = "Please answer the user_prompt about foster care policy and put a link to the relevant source at the end."

tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
)

# print(f"{'Total':<12}{'Instruction':<12}{'Story':<12}{'Summary':<12}")
# for pair in FILES:
#     with open(pair.format("story"), "r") as file:
#       story = "".join(file.readlines())
#     with open(pair.format("summary"), "r") as file:
#       summary = "".join(file.readlines())

#     # Count tokens
#     instruction_tokens = tokenizer(INSTRUCTION, return_tensors="pt")["input_ids"].shape[1]
#     story_tokens = tokenizer(story, return_tensors="pt")["input_ids"].shape[1]
#     summary_tokens = tokenizer(summary, return_tensors="pt")["input_ids"].shape[1]

#     # Print table of tokens
#     total_tokens = instruction_tokens + story_tokens + summary_tokens
#     print(f"{total_tokens:<12}{instruction_tokens:<12}{story_tokens:<12}{summary_tokens:<12}")

[('When does the policy change regarding the CAPS program transition take effect, and to which counties will it apply?', 'The policy change regarding the CAPS program transition took effect on July 17, 2017. Initially, it applied to 13 counties: Bibb, Brooks, Catoosa, Clarke, Colquitt, Cook, Douglas, Echols, Gilmer, Gordon, Lowndes, Murray, and Whitfield.\n\nPlease see the following link for more information: https://fostercaresystems.wustl.edu/foster/childcare-and-parent-services-caps-program-transition-department-early-care-and-learning'), ('What are the responsibilities and standards for caregivers in the Division of Family and Children Services (DFCS) when it comes to making decisions about age and developmentally appropriate activities for children in foster care?', "Caregivers in the Division of Family and Children Services (DFCS) are responsible for ensuring that children in foster care have ongoing opportunities to engage in age and developmentally appropriate activities, inclu

In [27]:
#yes
!pip install torch
from unsloth import FastLanguageModel
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch

if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please ensure you have a CUDA-enabled GPU and the correct drivers installed.")



max_seq_length = 8096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu118. CUDA = 7.5. CUDA Toolkit = 11.8.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [28]:
#yes
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [29]:
#yes
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
import os
print(os.listdir('/content/sample_data'))
#not needed

['anscombe.json', 'README.md', 'mnist_test.csv', 'california_housing_test.csv', 'california_housing_train.csv', 'mnist_train_small.csv']


In [ ]:
for i, pair in enumerate(FILES[:5]):
    print(f"Pair {i+1}:")
    print(f"  User Prompt: {pair[0]}")
    print(f"  System Response: {pair[1]}")

Pair 1:
  User Prompt: When does the policy change regarding the CAPS program transition take effect, and to which counties will it apply?
  System Response: The policy change regarding the CAPS program transition took effect on July 17, 2017. Initially, it applied to 13 counties: Bibb, Brooks, Catoosa, Clarke, Colquitt, Cook, Douglas, Echols, Gilmer, Gordon, Lowndes, Murray, and Whitfield.

Please see the following link for more information: https://fostercaresystems.wustl.edu/foster/childcare-and-parent-services-caps-program-transition-department-early-care-and-learning
Pair 2:
  User Prompt: What are the responsibilities and standards for caregivers in the Division of Family and Children Services (DFCS) when it comes to making decisions about age and developmentally appropriate activities for children in foster care?
  System Response: Caregivers in the Division of Family and Children Services (DFCS) are responsible for ensuring that children in foster care have ongoing opportunitie

In [30]:
#yes
from datasets import Dataset


# Prepare the dataset with your prompt template.
prompt = """Below is a question about foster care policy in Georgia. Please answer in the same style as me with a link to the relevant source at the end.

### user_prompt:
{}

### system_response:
{}

""" + tokenizer.eos_token

# Create the training data examples
finetuning_examples = []
for pair in FILES:
    user_prompt = pair[0].strip()
    system_response = pair[1].strip()
    finetuning_examples.append({
        "text": prompt.format(user_prompt, system_response)
    })

dataset = Dataset.from_list(finetuning_examples)

# Ensure your dataset is in a format that your model training process expects.
print(f"Number of examples: {len(dataset)}")
print(f"Example entry: {dataset[0]}")

Number of examples: 19
Example entry: {'text': 'Below is a question about foster care policy in Georgia. Please answer in the same style as me with a link to the relevant source at the end.\n\n### user_prompt:\nWhen does the policy change regarding the CAPS program transition take effect, and to which counties will it apply?\n\n### system_response:\nThe policy change regarding the CAPS program transition took effect on July 17, 2017. Initially, it applied to 13 counties: Bibb, Brooks, Catoosa, Clarke, Colquitt, Cook, Douglas, Echols, Gilmer, Gordon, Lowndes, Murray, and Whitfield.\n\nPlease see the following link for more information: https://fostercaresystems.wustl.edu/foster/childcare-and-parent-services-caps-program-transition-department-early-care-and-learning\n\n<|eot_id|>'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [31]:
#yes
def convert_to_conversations(example):
    """
    This function takes each 'text' entry and splits it into a structured
    format with 'role' (user/assistant) and 'content'.
    """
    text = example['text']

    # Split the prompt and response using the predefined markers in the prompt template
    user_prompt_start = "### user_prompt:\n"
    system_response_start = "### system_response:\n"

    # Find where the user prompt starts and system response starts
    user_prompt_index = text.find(user_prompt_start) + len(user_prompt_start)
    system_response_index = text.find(system_response_start)

    # Extract user prompt and system response
    user_prompt = text[user_prompt_index:system_response_index].strip()
    system_response = text[system_response_index + len(system_response_start):].strip()

    # Remove <|eot_id|> if present
    system_response = system_response.replace("<|eot_id|>", "").strip()

    # Return the conversation structure
    return {
        "conversations": [
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": system_response}
        ]
    }

# Apply the conversion function to each entry in the dataset
converted_dataset = dataset.map(convert_to_conversations, remove_columns=["text"])

# Check the output structure
print(converted_dataset[:5])

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

{'conversations': [[{'content': 'When does the policy change regarding the CAPS program transition take effect, and to which counties will it apply?', 'role': 'user'}, {'content': 'The policy change regarding the CAPS program transition took effect on July 17, 2017. Initially, it applied to 13 counties: Bibb, Brooks, Catoosa, Clarke, Colquitt, Cook, Douglas, Echols, Gilmer, Gordon, Lowndes, Murray, and Whitfield.\n\nPlease see the following link for more information: https://fostercaresystems.wustl.edu/foster/childcare-and-parent-services-caps-program-transition-department-early-care-and-learning', 'role': 'assistant'}], [{'content': 'What are the responsibilities and standards for caregivers in the Division of Family and Children Services (DFCS) when it comes to making decisions about age and developmentally appropriate activities for children in foster care?', 'role': 'user'}, {'content': "Caregivers in the Division of Family and Children Services (DFCS) are responsible for ensuring 

In [32]:
#yes
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(converted_dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/19 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [ ]:
dataset[5]["conversations"]

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
dataset[5]["text"]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [33]:
#yes
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 20, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/19 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [34]:
#yes
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [35]:
#yes
print("WANDB_DISABLED:", os.getenv("WANDB_DISABLED"))
trainer_stats = trainer.train()

WANDB_DISABLED: false


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 19 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 40
 "-____-"     Number of trainable parameters = 24,313,856


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.346400
2,2.175600
3,2.243400
4,2.139100
5,2.204700
6,1.972800
7,1.845000
8,1.670000
9,1.603900
10,1.594600


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [50]:
import pandas as pd
from unsloth.chat_templates import get_chat_template
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import nltk

# Ensure you have NLTK stopwords downloaded
nltk.download("stopwords")
from nltk.corpus import stopwords

# Step 1: Load the CSV file
file_path = '/content/data/Clean Missouri Data.csv'
df = pd.read_csv(file_path, encoding='MacRoman')

# Step 2: Define a function to search relevant policies based on the user's query using cosine similarity
def search_relevant_policies(query, df, top_n=10):
    # Convert policies into a TF-IDF matrix
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['Content'])

    # Get the query as a TF-IDF vector
    query_vector = tfidf.transform([query])

    # Calculate cosine similarity between query and policies
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get the top N relevant policies
    top_indices = cosine_sim.argsort()[-top_n:][::-1]
    relevant_policies = df.iloc[top_indices]

    return relevant_policies

# Step 3: User's query
user_query = "What is the allowance for infants that the state will pay for?"

# Step 4: Retrieve relevant policies based on the query
relevant_policies = search_relevant_policies(user_query, df)

# Step 5: Combine the relevant policies with the user's query for the model
formatted_policies = []
for index, row in relevant_policies.iterrows():
    formatted_policy = f"Title: {row['Title']}\nTerritory: {row['Territory']}\nType: {row['Type']}\nYear: {row['Year']}\nCategory: {row['Category']}\nFrom: {row['From']}\nTo: {row['To']}\nContent: {row['Content']}\nLink: {row['Link to Content']}\n"
    formatted_policies.append(formatted_policy)
relevant_policy_text = "\n\n".join(formatted_policies)

# Messages with relevant policies for the model
messages_with_relevant_policies = [
    {"role": "system", "content": relevant_policy_text},
    {"role": "user", "content": user_query},
]

# Step 6: Apply chat template and tokenize
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)
inputs = tokenizer.apply_chat_template(
    messages_with_relevant_policies,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

FastLanguageModel.for_inference(model)
outputs = model.generate(input_ids=inputs, max_new_tokens=256, use_cache=True, temperature=1.5, min_p=0.1)

# Step 7: Decode the output
generated_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(generated_response)

# Step 8: Rank the top 10 policies using SBERT for the final link
# Load SBERT model
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose another SBERT model if desired

# Encode the generated response using SBERT
response_embedding = model_sbert.encode(generated_response, convert_to_tensor=True)

# Encode each policy in the top 10 list
policy_embeddings = model_sbert.encode(relevant_policies['Content'].tolist(), convert_to_tensor=True)

# Calculate cosine similarities between the generated response and each policy embedding
cosine_similarities = util.cos_sim(response_embedding, policy_embeddings).flatten()

# Identify the policy with the highest SBERT cosine similarity score
most_relevant_index = cosine_similarities.argmax().item()
most_relevant_link = relevant_policies.iloc[most_relevant_index]['Link to Content']

# Print the link to the most relevant source
print("Link to the most relevant source:", most_relevant_link)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Title: MO :: Section 4, Chapter 12 (Financial Considerations), Subsection 1 ‚Äì Referral Process :: 12.1.2 Service Needs
Territory: Missouri
Type: Rules
Year: 2023, 2022, 2021, 2020, 2019
Category: Services while in care - Financial, Services while in care - Medical, Subsidies - Other
From: nan
To: nan
Content: The Case Manager/Children‚Äôs Service Worker will determine services needed by the child for which CD will pay.¬† Services may include: Maintenance Infant allowance:¬† A child between the ages of 0 ‚Äì 36 months, in qualifying placement types, is eligible for an additional $50 per month to help meet the specialized needs of infants, such as diapers, formula, clothing, and supplies Children eligible for this allowance include licensed foster homes, relative and kinship homes, children in CD custody in court ordered placements, and CYAC.¬† Infants in residential care facilities are not eligible to receive the 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Link to the most relevant source: https://fostercaresystems.wustl.edu/foster/mo-section-4-chapter-12-financial-considerations-subsection-1-referral-process-1212-service


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the heart of Paris, stands tall among the city's historic and cultural landmarks. This iron structure, standing at an impressive 324 meters high, offers breathtaking views of the City of Light's iconic landscape. The Eiffel Tower was built for the 1889 World's Fair and has since become a symbol of French engineering and culture.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>